<a href="https://colab.research.google.com/github/parisasalma/FYP/blob/master/93_2_mc%2Bbabi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
from gensim.parsing import PorterStemmer
from gensim.models import Word2Vec, Phrases, phrases, KeyedVectors
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk import tokenize
from __future__ import print_function
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Activation, Dense, Permute, Dropout,multiply
from keras.layers import add, dot, concatenate, BatchNormalization
from keras.layers import LSTM
from keras.utils.data_utils import get_file
from keras.preprocessing.sequence import pad_sequences
from functools import reduce
import tarfile
import numpy as np
import re
import pandas as pd

import pickle
from keras.models import load_model
import re
from collections import defaultdict
import string
import pandas as pd
import nltk

Using TensorFlow backend.


In [0]:
import re
from collections import defaultdict
import string
import pandas as pd
import nltk
file=open('/content/gdrive/My Drive/FYP2/mcc40.txt','r')
#file = open("data/MCTest/mc160.train.txt", "r") 
obj= (file.read())
splitss1=(obj.split('***************************************************'))
s=(len(splitss1))
df_dd1 = pd.DataFrame(index=range(s-1),
                   columns=['story','q','A'])

In [0]:
def tokenize(sent):
    '''Return the tokens of a sentence including punctuation.
    >>> tokenize('Bob dropped the apple. Where is the apple?')
    ['Bob', 'dropped', 'the', 'apple', '.', 'Where', 'is', 'the', 'apple', '?']
    '''
    return [x.strip() for x in re.split(r'(\W+)?', sent) if x.strip()]


In [0]:
for i in range(1,len(splitss1)-1):
  ############ story reading ###########
  story=(splitss1[i].split('1:')[0].strip())
  df_dd1['story'][i]=tokenize(story)
  #print (story)
  ############ question reading ######### 
  q=(splitss1[i].split('*D)')[0].strip())
  position=(q.find('1:'))
  q=q[position+2:]
  pos_colon=(q.find(':'))
  question=(q[pos_colon+1:])
  df_dd1['q'][i]=tokenize(question) 
  #print (question)
  
  ########### answer reading ###########
  ans_pos=splitss1[i].find('*D)')
  ans=splitss1[i][ans_pos+3:]
  df_dd1['A'][i]=ans.rstrip('\n')
  #print (ans)

/usr/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [0]:
#print (df_dd1)

In [0]:
def parse_stories(lines, only_supporting=False):
    '''Parse stories provided in the bAbi tasks format
    If only_supporting is true, only the sentences
    that support the answer are kept.
    '''
    flatten = lambda data: reduce(lambda x, y: x + y, data)
    all_stories=[]
    all_ques=[]
    all_ans=[]
    data = []
    story = []
    #print ('liness ',lines)
    for line in lines:
        line = line.decode('utf-8').strip()
        nid, line = line.split(' ', 1)
        nid = int(nid)
        if nid == 1:
            story = []
        if '\t' in line:
            q, a, supporting = line.split('\t')
            q = tokenize(q)
            if only_supporting:
                # Only select the related substory
                supporting = map(int, supporting.split())
                substory = [story[i - 1] for i in supporting]
            else:
                # Provide all the substories
                substory = [x for x in story if x]
            data.append((substory, q, a))
            story.append('')
            all_stories.append(flatten(substory))
            all_ques.append(q)
            all_ans.append(a)
        else:
            sent = tokenize(line)
            story.append(sent)
           
    return data, all_stories, all_ques, all_ans

In [0]:
def get_stories(f, only_supporting=False, max_length=None):
    '''Given a file name, read the file,
    retrieve the stories,
    and then convert the sentences into a single story.
    If max_length is supplied,
    any stories longer than max_length tokens will be discarded.
    '''
    data,stories,ques,ans = parse_stories(f.readlines(), only_supporting=only_supporting)
    flatten = lambda data: reduce(lambda x, y: x + y, data)
    data = [(flatten(story), q, answer) for story, q, answer in data
            if not max_length or len(flatten(story)) < max_length]
    return data,stories,ques,ans

In [0]:
# Downloading Dataset
try:
    path = get_file('babi-tasks-v1-2.tar.gz',
                    origin='https://s3.amazonaws.com/text-datasets/'
                           'babi_tasks_1-20_v1-2.tar.gz')
except:
    print('Error downloading dataset, please download it manually:\n'
          '$ wget http://www.thespermwhale.com/jaseweston/babi/tasks_1-20_v1-2'
          '.tar.gz\n'
          '$ mv tasks_1-20_v1-2.tar.gz ~/.keras/datasets/babi-tasks-v1-2.tar.gz')
    raise

11747328/11745123 [==============================] - 2s 0us/step


In [0]:
challenges = {
    # QA1 with 10,000 samples
    'single_supporting_fact_10k': 'tasks_1-20_v1-2/en-10k/qa1_'
                                  'single-supporting-fact_{}.txt',
    # QA2 with 10,000 samples
    'two_supporting_facts_10k': 'tasks_1-20_v1-2/en-10k/qa2_'
                                'two-supporting-facts_{}.txt',
}
challenge_type = 'single_supporting_fact_10k'
challenge = challenges[challenge_type]


In [0]:
#print('Extracting stories for the challenge:', challenge_type)
with tarfile.open(path) as tar:
    train_stories,train_story,train_ques,train_ans = get_stories(tar.extractfile(challenge.format('train')))
    test_stories,test_story,test_ques,test_ans = get_stories(tar.extractfile(challenge.format('test')))

/usr/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [0]:
for i in range(1,len(df_dd1)-1):
  train_story.append(df_dd1['story'][i])
  train_ques.append(df_dd1['q'][i])
  train_ans.append(df_dd1['A'][i])

In [0]:
'''wv_model=Word2Vec.load('/content/gdrive/My Drive/FYP2/Saved Models/wv_model.model')
with open('/content/gdrive/My Drive/FYP2/Saved Models/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)'''

"wv_model=Word2Vec.load('/content/gdrive/My Drive/FYP2/Saved Models/wv_model.model')\nwith open('/content/gdrive/My Drive/FYP2/Saved Models/tokenizer.pickle', 'rb') as handle:\n    tokenizer = pickle.load(handle)"

In [0]:
sentences=[]
MAX_NB_WORDS = 20000
for i in train_stories + test_stories:
  sentences.append(i[0])
  sentences.append(i[1])

In [0]:
for i in range(1,len(df_dd1)-1):
  sentences.append(df_dd1['story'][i])
  sentences.append(df_dd1['q'][i])
  sentences.append(df_dd1['A'][i])

In [0]:
google_wv = KeyedVectors.load_word2vec_format('/content/gdrive/My Drive/FYP2/GoogleNews-vectors-negative300.bin.gz', binary=True,limit=500000)
wv_model = Word2Vec(size=300, min_count=1, iter=10)
wv_model.build_vocab(sentences)
wv_model.build_vocab([list(google_wv.vocab.keys())], update=True)
wv_model.intersect_word2vec_format("/content/gdrive/My Drive/FYP2/GoogleNews-vectors-negative300.bin.gz",binary=True, lockf=1.0)
wv_model.train(sentences,total_examples=len(sentences), epochs=300)

(101845054, 144102900)

In [0]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(sentences)

input_train = tokenizer.texts_to_sequences(train_story)
answers_train=tokenizer.texts_to_sequences(train_ans)
queries_train=tokenizer.texts_to_sequences(train_ques)

inputs_test = tokenizer.texts_to_sequences(test_story)
answers_test=tokenizer.texts_to_sequences(test_ans)
queries_test=tokenizer.texts_to_sequences(test_ques)
ans_train_oldcopy=answers_train

In [0]:
kkk=np.array(answers_train)
print (kkk.shape[0])
answers_train=np.zeros(shape=(kkk.shape[0],1),dtype=int)
ind=0
for i in ans_train_oldcopy:
  #print (ind)
  #print (i[0])
  answers_train[ind,0]=i[0]
  ind+=1
#print (answers_train)

10111


In [0]:
word_idx = tokenizer.word_index
vocab_size=len(word_idx)+1

glove_vocab=wv_model.wv.vocab
embeddingMatrix = np.zeros((len(word_idx) + 1, 300))
for word, i in word_idx.items():
  if word in glove_vocab:
    embeddingVector = wv_model[word]
    embeddingMatrix[i] = embeddingVector

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [0]:
print (len(word_idx)+1)

1362


In [0]:
#vocab_size = len(vocab) + 1
story_maxlen = max(map(len, (x for x, _, _ in train_stories + test_stories)))
query_maxlen = max(map(len, (x for _, x, _ in train_stories + test_stories)))

l=[]
for i in range(1,s-1):
  l.append(len(df_dd1['story'][i]))
l=np.array(l)
mc_max= (np.max(l))  
story_maxlen=max(mc_max,story_maxlen)
print (story_maxlen)

l=[]
for i in range(1,s-1):
  l.append(len(df_dd1['q'][i]))
l=np.array(l)
mc_max= (np.max(l))  
query_maxlen=max(mc_max,query_maxlen)
print (query_maxlen)

398
17


In [0]:
inputs_train=pad_sequences(input_train,maxlen=story_maxlen)
queries_train=pad_sequences(queries_train,maxlen=query_maxlen)
inputs_test=pad_sequences(inputs_test,maxlen=story_maxlen)
queries_test=pad_sequences(queries_test,maxlen=query_maxlen)

In [0]:
##############################################################################
#                                Input Layers                                #
##############################################################################
input_sequence = Input((story_maxlen,))
question_sequence = Input((query_maxlen,))

In [0]:
##############################################################################
#                                Story Model                                 #
##############################################################################

Story_Model_Embed=Embedding(input_dim=vocab_size,weights=[embeddingMatrix],
                              output_dim=300, input_length=story_maxlen)(input_sequence)
Story_Model_lstm1=LSTM(300,return_sequences=True,dropout=0.4)(Story_Model_Embed)
#Story_Model_lstm2=LSTM(300,return_sequences=True,dropout=0.4)(Story_Model_lstm1)
#Story_Model_lstm3=LSTM(300,return_sequences=True,dropout=0.4)(Story_Model_lstm1)
Story_Model_Norm=BatchNormalization()(Story_Model_lstm1)
Story_Model_Attention=multiply([Story_Model_Embed, Story_Model_Norm])

input_encoder_m=Model(input=[input_sequence], output=Story_Model_Attention)
input_encoder_m.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 398)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 398, 300)     408600      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 398, 300)     721200      embedding_1[0][0]                
__________________________________________________________________________________________________
batch_normalization_1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("mu...)`
  # Remove the CWD from sys.path while we load stuff.


In [0]:
##############################################################################
#                           Story embedded to question                       #
##############################################################################
Story_question_Embed=Embedding(input_dim=vocab_size,
                              output_dim=query_maxlen)(input_sequence)
Story_question_lstm1=LSTM(query_maxlen,return_sequences=True,dropout=0.4)(Story_question_Embed)
#Story_question_lstm2=LSTM(query_maxlen,return_sequences=True,dropout=0.4)(Story_question_lstm1)
#Story_question_lstm3=LSTM(query_maxlen,return_sequences=True,dropout=0.4)(Story_question_lstm1)
Story_question_Norm=BatchNormalization()(Story_question_lstm1)
Story_question_Attention=multiply([Story_question_Embed, Story_question_Norm])

input_encoder_c=Model(input=[input_sequence], output=Story_question_Attention)
input_encoder_c.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 398)          0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 398, 17)      23154       input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   (None, 398, 17)      2380        embedding_2[0][0]                
__________________________________________________________________________________________________
batch_normalization_2 (BatchNor (None, 398, 17)      68          lstm_2[0][0]                     
__________________________________________________________________________________________________
multiply_2

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("mu...)`
  if __name__ == '__main__':


In [0]:
##############################################################################
#                                Question Model                              #
##############################################################################

Question_Model_Embed=Embedding(input_dim=vocab_size,weights=[embeddingMatrix],
                              output_dim=300, input_length=query_maxlen)(question_sequence)
Question_Model_lstm1=LSTM(300,return_sequences=True,dropout=0.4)(Question_Model_Embed)
#Question_Model_lstm2=LSTM(300,return_sequences=True,dropout=0.4)(Question_Model_lstm1)
#Question_Model_lstm3=LSTM(300,return_sequences=True,dropout=0.4)(Question_Model_lstm1)
Question_Model_Norm=BatchNormalization()(Question_Model_lstm1)
Question_Model_Attention=multiply([Question_Model_Embed, Question_Model_Norm])

question_encoder=Model(input=[question_sequence], output=Question_Model_Attention)
question_encoder.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 17)           0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 17, 300)      408600      input_2[0][0]                    
__________________________________________________________________________________________________
lstm_3 (LSTM)                   (None, 17, 300)      721200      embedding_3[0][0]                
__________________________________________________________________________________________________
batch_normalization_3 (BatchNor (None, 17, 300)      1200        lstm_3[0][0]                     
__________________________________________________________________________________________________
multiply_3

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("mu...)`
  # Remove the CWD from sys.path while we load stuff.


In [0]:
# encode input sequence and questions (which are indices)
# to sequences of dense vectors
input_encoded_m = input_encoder_m(input_sequence)
input_encoded_c = input_encoder_c(input_sequence)
question_encoded = question_encoder(question_sequence)

print (input_encoded_m)
print (input_encoded_c)
print (question_encoded)

Tensor("model_1/multiply_1/mul:0", shape=(?, 398, 300), dtype=float32)
Tensor("model_2/multiply_2/mul:0", shape=(?, 398, 17), dtype=float32)
Tensor("model_3/multiply_3/mul:0", shape=(?, 17, 300), dtype=float32)


In [0]:
# compute a 'match' between the first input vector sequence
# and the question vector sequence
# shape: `(samples, story_maxlen, query_maxlen)`
match = dot([input_encoded_m, question_encoded], axes=(2, 2))
print (match)
match = Activation('softmax')(match)

print (match)

Tensor("dot_1/MatMul:0", shape=(?, 398, 17), dtype=float32)
Tensor("activation_1/truediv:0", shape=(?, 398, 17), dtype=float32)


In [0]:
##################################################################
#Attention?????
##################################################################

# add the match matrix with the second input vector sequence
response = add([match, input_encoded_c])  # (samples, story_maxlen, query_maxlen)
print (response)
response = Permute((2, 1))(response)  # (samples, query_maxlen, story_maxlen)
print (response)

###################################################################
#get importance of each sentence with respect to question
##################################################################

Tensor("add_1/add:0", shape=(?, 398, 17), dtype=float32)
Tensor("permute_1/transpose:0", shape=(?, 17, 398), dtype=float32)


In [0]:
# concatenate the match matrix with the question vector sequence
answer = concatenate([response, question_encoded])
print (answer)

Tensor("concatenate_1/concat:0", shape=(?, 17, 698), dtype=float32)


In [0]:
# the original paper uses a matrix multiplication for this reduction step.
# we choose to use a RNN instead.
answer = LSTM(32)(answer)  # (samples, 32)
print (answer)

Tensor("lstm_4/TensorArrayReadV3:0", shape=(?, 32), dtype=float32)


In [0]:
print (vocab_size)

1362


In [0]:
# one regularization layer -- more would probably be needed.
answer = Dropout(0.3)(answer)

answer = Dense(vocab_size)(answer)  # (samples, vocab_size)
# we output a probability distribution over the vocabulary
answer = Activation('softmax')(answer)


In [0]:
# build the final model
model = Model([input_sequence, question_sequence], answer)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 398)          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 17)           0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, 398, 300)     1131000     input_1[0][0]                    
__________________________________________________________________________________________________
model_3 (Model)                 (None, 17, 300)      1131000     input_2[0][0]                    
__________________________________________________________________________________________________
dot_1 (Dot

In [0]:
############################################
#              training                    #
############################################
model.fit([(inputs_train), (queries_train)], np.array(answers_train),
          batch_size=50,
          epochs=100,
          validation_data=([(inputs_test), (queries_test)], np.array(answers_test)))

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 10111 samples, validate on 1000 samples
Epoch 1/100
10111/10111 [==============================] - 370s 37ms/step - loss: 3.0345 - acc: 0.1603 - val_loss: 1.8225 - val_acc: 0.1870
Epoch 2/100
10111/10111 [==============================] - 366s 36ms/step - loss: 1.8700 - acc: 0.1634 - val_loss: 1.8023 - val_acc: 0.1570
Epoch 3/100
10111/10111 [==============================] - 365s 36ms/step - loss: 1.8524 - acc: 0.1661 - val_loss: 1.8009 - val_acc: 0.1820
Epoch 4/100
10111/10111 [==============================] - 362s 36ms/step - loss: 1.8498 - acc: 0.1629 - val_loss: 1.7928 - val_acc: 0.1710
Epoch 5/100
10111/10111 [==============================] - 359s 35ms/step - loss: 1.8459 - acc: 0.1728 - val_loss: 1.7918 - val_acc: 0.1860
Epoch 6/100
10111/10111 [==============================] - 375s 37ms/step - loss: 1.8440 - acc: 0.1663 - val_loss: 1.7939 - v

In [0]:
model.save("/content/gdrive/My Drive/FYP2/mc+babi10111_93.6%.h5")

In [0]:
predictions=model.predict([np.array(inputs_test), np.array(queries_test)], batch_size=None, verbose=0, steps=None)

In [0]:
answer_seq=[]
for i in range(len(test_stories)):
  ind=np.argmax(predictions[i])
  answer_seq.append(ind)

In [0]:
print (np.array(answers_test).shape)
print (np.sum(np.array(answer_seq)[:,None]==np.array(answers_test))) #

(1000, 1)
932


In [0]:
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))
def sequence_to_text(list_of_indices):
    words = [reverse_word_map.get(letter) for letter in list_of_indices]
    return(words)

In [0]:
print (answers_train)

[[ 14]
 [ 17]
 [ 17]
 ...
 [967]
 [435]
 [321]]


In [0]:
answers= sequence_to_text(np.array(answer_seq))
answers_real=sequence_to_text(np.array(answers_test)[:,0])
#print (answers_real[0])

hallway


In [0]:
for i in range(1000):
  print ('predicted',answers[i])
  print ('real',answers_real[i])

predicted hallway
real hallway
predicted bathroom
real bathroom
predicted kitchen
real kitchen
predicted hallway
real hallway
predicted kitchen
real kitchen
predicted hallway
real hallway
predicted garden
real garden
predicted hallway
real hallway
predicted office
real office
predicted office
real office
predicted kitchen
real kitchen
predicted garden
real garden
predicted office
real office
predicted bedroom
real bedroom
predicted bedroom
real bedroom
predicted kitchen
real kitchen
predicted garden
real garden
predicted kitchen
real kitchen
predicted office
real office
predicted hallway
real kitchen
predicted hallway
real hallway
predicted office
real office
predicted office
real office
predicted hallway
real hallway
predicted garden
real bedroom
predicted bedroom
real bedroom
predicted bedroom
real bedroom
predicted kitchen
real kitchen
predicted kitchen
real kitchen
predicted kitchen
real kitchen
predicted hallway
real hallway
predicted office
real office
predicted kitchen
real kitc